# Centralized version

In [ ]:
#%load_ext autoreload
#%autoreload 2

import importlib
import random
import string
from river import sketch
from river import stream
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

#epilson influencia no tamanho da janela = 1/epilson
#hh = sketch.HeavyHitters(fading_factor=1, epsilon=0.00025, support=0.00025) # Para window 4000
#hh = sketch.HeavyHitters(fading_factor=1, epsilon=0.0002, support=0.0002) # Para window 5000
hh = sketch.HeavyHitters(fading_factor=1, epsilon=0.0005, support=0.0005) # Para window 2000
##hh = sketch.HeavyHitters(fading_factor=1, epsilon=0.001, support=0.001) # Para window 1000
#hh = sketch.HeavyHitters(fading_factor=1, epsilon=0.000333, support=0.000333) # Para window 3000
#hh = sketch.HeavyHitters(fading_factor=1, epsilon=0.000125, support=0.000125) # Para window 8000
#hh = sketch.HeavyHitters(fading_factor=1, epsilon=0.0001, support=0.0001) # Para window 10000

In [ ]:
diretorio = "/home/paula/ecml2024/communication_data_june"

nome_coluna = 'ANumber'

most_common_iter_a = []

df_bucket_a = pd.DataFrame(columns=['ANumber', 'Count', 'DateTime'])
colunas = ['ANumber','BNumber','DateTime','Action','Result','CountryCode']
window = 1

# Verificar se o diretório existe antes de tentar abri-lo
if os.path.exists(diretorio):
    arquivos = [arquivo for arquivo in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, arquivo))]
    arquivos = sorted(arquivos)
    
    for arquivo in arquivos:
        posicao = arquivo.find('.ipynb_checkpoints')
        if (posicao != 0):
            # abrir arquivo
            X_y = stream.iter_csv(diretorio + "/" + arquivo, delimiter=',', fieldnames=colunas)
            for x, y in X_y:
                res = x.__len__()
                if res > 0:
                    hh = hh.update(x[nome_coluna])
                    if hh._n % hh._bucket_width == 0:
                        data_obj = datetime.strptime(x['DateTime'], '%Y%m%d%H%M%S')
                        most_common_iter_a = hh.most_common(5)
                        data = x['DateTime']
                        df_new_row = pd.DataFrame(most_common_iter_a, columns=['ANumber', 'Count'])
                        df_new_row['DateTime'] = data
                        df_new_row['Window'] = window
                        # Excluindo colunas vazias ou todas NA do df_new_row antes de concatenar
                        df_new_row = df_new_row.dropna(axis=1, how='all')
                        df_bucket_a = pd.concat([df_bucket_a, df_new_row], axis = 0)
                        window += 1
                        #hh = sketch.HeavyHitters(fading_factor=1, epsilon=0.00025, support=0.00025)
else:
    print("The directory does not exist.")

In [ ]:
print(df_bucket_a)

In [ ]:
# Top 10 - Mais comuns por janela
#print(df_bucket_a)

# Salvar DataFrame como arquivo CSV incluindo índices
df_bucket_a.to_csv('/home/paula/ecml2024/results_centralized/result_data_june_4000_updated.csv', index=False)

In [ ]:
from submit_email import send_email
window = 4000
send_email("centralized lossy counting JUNE" + " " + str(window))